# Data Labelling

In [20]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.model_selection import cross_validate
from sklearn.linear_model import LinearRegression

In [21]:
!pwd
!

/root/code/FlorenceBoutin/GI_disease_detection


### Import the train, test, val directory paths which will be used by the image_dataset_from_directory function

In [40]:
train_directory = '/root/code/FlorenceBoutin/raw_data/Gi_disease_data/train'
test_directory = '/root/code/FlorenceBoutin/raw_data/Gi_disease_data/test'
val_directory = '/root/code/FlorenceBoutin/raw_data/Gi_disease_data/val'



In [41]:
# from keras.preprocessing.image import ImageDataGenerator
# train_normal_directory = ImageDataGenerator(rescale=1./255)

In [42]:
import tensorflow as tf


train = tf.keras.utils.image_dataset_from_directory(
    train_directory,
    labels= "inferred",
    color_mode='rgb',
    image_size=(224, 224),
    )

Found 3200 files belonging to 4 classes.


In [43]:
test = tf.keras.utils.image_dataset_from_directory(
    test_directory,
    labels= "inferred",
    color_mode='rgb',
    image_size=(224, 224)
    )

Found 800 files belonging to 4 classes.


In [44]:
val = tf.keras.utils.image_dataset_from_directory(
    val_directory,
    labels= "inferred",
    color_mode='rgb',
    image_size=(224, 224)
    )

Found 2000 files belonging to 4 classes.


In [45]:
pip install opencv-python


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.8/61.8 MB 10.5 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
import cv2

def load_images(folder):
    """
    Enter a folder directory with images to load.
    """
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename))
        img_rgb = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        if img_rgb is not None:
            images.append(img_rgb)
    return images
